In [1]:
#https://github.com/Jcharis/Natural-Language-Processing-Tutorials/blob/master/Training%20the%20Named%20Entity%20Recognizer%20in%20SpaCy.ipynb
#https://spacy.io/usage/training#ner
import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy import displacy
import pandas as pd
import numpy as np
import re

In [2]:
example = "Heavy rain closes roads in Christchurch, causes flooding."
for match in re.finditer('Christchurch', example):
    print(match.start(), match.end())

27 39


In [42]:
LABEL = "CAUSE"

TRAIN_DATA = [
("Heavy rain closes roads in Christchurch, causes flooding",{"entities":[(0, 10, LABEL), (11, 23,"EVENT"), (27, 39,"GPE"), (48, 56,LABEL)]}),
("Deadly storm causes travel chaos amid evacuations and widespread flooding",{"entities":[(65, 73,LABEL), (20, 32, "EVENT")]}),
("Roads under water and fallen trees, as storm wreaks havoc on region",{"entities":[(0, 17,"EVENT"), (22, 34,"EVENT"), (39, 44, LABEL)]}),
("Tree fall in Rotorua kills woman in car",{"entities":[(0, 9, LABEL), (13, 20,"GPE"), (21, 26,"EVENT")]}),
("Thames Coast road severely damaged in wake of storm",{"entities":[(0, 17,"LOC"), (18, 34,"EVENT"), (46, 51, LABEL)]}),
("Flooding closes roads, inundates homes in Auckland",{"entities":[(0, 8, LABEL), (9, 21,"EVENT"), (42, 50,"GPE")]}),
("Wild weather batters the Bay of Plenty",{"entities":[(0, 12, LABEL), (13, 20,"EVENT"), (25, 38,"GPE")]}),
("Storm hit power supplies almost restored",{"entities":[(0, 5, LABEL), (6, 24,"EVENT"), (32, 40,"EVENT")]}),
("State of emergency declared in Nelson Tasman District",{"entities":[(0, 27,"EVENT"), (31, 37,"GPE"), (38, 37,"GPE")]}),
("Several Christchurch residents evacuate as Heathcote River threatens to overflow",{"entities":[(8, 20,"GPE"), (31, 39,"EVENT"), (43, 58,"LOC"), (72, 80, LABEL)]}),
("Cyclone Gita: Wild weather grounds flights, knocks down power pole in Wellington",{"entities":[(0, 12, LABEL1), (27, 42,"EVENT"), (44, 66,"EVENT"), (70, 80,"GPE")]}),
("Former cyclone Gita crosses New Zealand, leaving destruction in its wake",{"entities":[(7, 19, LABEL1), (28, 39,"EVENT")]}),
("Cyclone Gita aftermath: What you need to know",{"entities":[(0, 12, LABEL1)]}),
("Houses evacuated at Kāpiti Coast as high tide and heavy rain cause flooding",{"entities":[(7, 16,"EVENT"), (20, 32,"GPE"), (36, 45,LABEL), (50, 60,LABEL), (67, 75,LABEL)]}),
("Takaka Hill road partially reopens, to the relief of those trapped in Golden Bay after Gita",{"entities":[(0, 16,"LOC"), (17, 34,"EVENT"), (70, 80,"LOC"), (87, 91,LABEL1)]}),
("State Highway 1 through Kaikōura likely to remain closed until end of the week",{"entities":[(0, 15,"FAC"),(24, 32,"GPE"),(43, 56,"EVENT"),(63, 78,"DATE")]}),
("Takaka Hill road to fully re-open from next weekend",{"entities":[(0, 16,"LOC"), (20, 33,"EVENT"), (39, 51,"DATE")]}),
("Never seen it like this: Homes flooded, roads closed by storm",{"entities":[(25, 38,"EVENT"), (40, 52,"EVENT"), (56, 61, LABEL)]}),
("Flights disrupted and roads closed as high winds continue to batter Wellington",{"entities":[(0, 17,"EVENT"), (22, 34,"EVENT"), (38, 48, LABEL), (68, 78,"GPE")]}),
("Local state of emergency declared, homes evacuated as heavy rain, gales thrash West Coast",{"entities":[(0, 33,"EVENT"), (41, 50,"EVENT"), (54, 64, LABEL),(79, 89,"GPE")]}),
("As it happened: West Coast bridge washed away as rivers reach record levels",{"entities":[(16, 33,"FAC"), (34, 45,"EVENT")]}),
("Coasters on clean-up duty have till Sunday before the rain returns",{"entities":[(36, 42,"DATE")]}),
("Vital Waiho River bridge reopens, reconnecting a recovering West Coast",{"entities":[(6, 24,"FAC"),(25, 32,"EVENT"),(60, 70,"GPE")]}),
("Tornado clean-up begins as Key heads to destruction zone",{"entities":[(8, 23,"EVENT"), (0, 7,LABEL)]}),
("Rotorua tornado: Calves tossed into the air",{"entities":[(0, 7,"GPE"), (8, 15, LABEL), (17, 43, "EVENT")]}),
("Snow spotted in central Wellington",{"entities":[(0, 12,"EVENT"), (24, 34,"GPE")]}),
("Wellington's coldest day on record",{"entities":[(0, 10,"GPE"), (13, 34,"EVENT")]}),
("Region freezes with big chill",{"entities":[(0, 14,"EVENT")]}),
("Snow forces Wellington schools to close",{"entities":[(12, 22,"GPE"), (23, 39,"EVENT")]}),
("One person dead after crash involving car and cyclist on SH2 north of Wellington",{"entities":[(0, 3,"CARDINAL"), (11, 15,"EVENT"), (22, 27,LABEL), (57, 66,"FAC"),(70, 80,"GPE")]}),
("Strong 5.8 magnitude quake near Levin",{"entities":[(7, 10,"CARDINAL"), (21, 26,"EVENT"), (32, 37,"GPE")]}),
("Eight people hurt, one seriously, in Southland crash",{"entities":[(0, 5,"CARDINAL"), (13, 17,"EVENT"), (19, 22,"CARDINAL"), (37, 46,"GPE")]}),
("Vital Waiho River bridge reopens, reconnecting a recovering West Coast",{"entities":[(6, 24,"FAC"), (25, 32,"EVENT"), (60, 70,"GPE")]}),
("Warnings as polar blast hits country",{"entities":[(12, 28,"EVENT")]}),
("Five crashes on Auckland's Southwestern Motorway",{"entities":[(5, 12,"EVENT"), (16, 24,"GPE"), (27, 48,"FAC")]}),
("Auckland motorway closure: drivers asked to avoid area",{"entities":[(0, 8,"FAC"), (18, 25,"EVENT"), (27, 54,"EVENT")]}),
("Major weather disruption around NZ",{"entities":[(32, 34,"GPE")]}),
]




# ("Big freeze: Worst over but disruption remains",{"entities":[(0, 45,"GPE")]}),
# ("Icy blast moves up from south",{"entities":[(0, 29,"GPE")]}),
# ("Wintry blast brings country to a standstill",{"entities":[(0, 43,"GPE")]}),
# ("Christchurch's new white zone",{"entities":[(0, 29,"GPE")]}),
# ("Worst over but snow could hit hills",{"entities":[(0, 35,"GPE")]}),
# ("Wellington's 'once in a lifetime' polar blast",{"entities":[(0, 45,"GPE")]}),
# ("Another three days of snow for Wellington",{"entities":[(0, 41,"GPE")]}),
# ("Wellington snow most severe since 1976",{"entities":[(0, 38,"GPE")]}),
# ("Black ice a risk as weather eases",{"entities":[(0, 33,"GPE")]}),
# ("Snow closes Nelson roads",{"entities":[(0, 24,"GPE")]}),
# ("Snow showers will ease tomorrow",{"entities":[(0, 31,"GPE")]}),
# ("As it happened: West Coast bridge washed away as rivers reach record levels",{"entities":[(0, 75,"GPE")]}),
# ("Live updates: West Coast flooding day 2",{"entities":[(0, 39,"GPE")]}),
# ("Coasters on clean-up duty have till Sunday before the rain returns",{"entities":[(0, 66,"GPE")]}),
# ("Live updates: Nelson fire",{"entities":[(0, 25,"GPE")]}),
# ("Nelson fire: All you need to know",{"entities":[(0, 33,"GPE")]}),
# ("Nelson fire: Fire contained, evacuated residents return home",{"entities":[(0, 60,"GPE")]}),
# ("Giant hail stones hammer Timaru as storm moves up the country",{"entities":[(0, 61,"GPE")]}),
# ("Wellington, Central North Island 5.8 magnitude earthquake had no known links to major fault lines",{"entities":[(0, 97,"GPE")]}),


In [2]:
model = 'en_core_web_sm'
output_dir=Path("/home/rangika/PythonCode/TrainedModel2.0")
n_iter=20

In [5]:
random.seed(0)
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")

Loaded model 'en_core_web_sm'


In [6]:
if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
else:
    ner = nlp.get_pipe("ner")

In [7]:
ner.add_label(LABEL)

In [8]:
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    # only train NER
    with nlp.disable_pipes(*other_pipes) and warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

Losses {'ner': 61.66876282527873}
Losses {'ner': 57.317302166021676}
Losses {'ner': 47.51047554269553}
Losses {'ner': 49.49937161026551}
Losses {'ner': 47.20177820772264}
Losses {'ner': 53.16326780825594}
Losses {'ner': 46.26461603723419}
Losses {'ner': 40.15023700479276}
Losses {'ner': 48.4969237564662}
Losses {'ner': 45.00271484762213}
Losses {'ner': 45.38826974946349}
Losses {'ner': 41.99459004196251}
Losses {'ner': 44.965490107017104}
Losses {'ner': 36.78450473284465}
Losses {'ner': 45.986399406530836}
Losses {'ner': 47.78294669950452}
Losses {'ner': 43.830482330173254}
Losses {'ner': 40.69150386082765}
Losses {'ner': 42.307795716183875}
Losses {'ner': 40.49295756596812}


In [9]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Thames Coast road', 'LOC'), ('severely damaged', 'EVENT')]
Entities [('Heavy rain', 'CAUSE'), ('closes roads', 'EVENT'), ('Christchurch', 'GPE')]
Entities [('Roads under water', 'EVENT'), ('fallen trees', 'EVENT')]
Entities [('travel chaos', 'EVENT')]
Entities [('closes roads', 'EVENT'), ('Auckland', 'GPE')]
Entities [('Tree fall', 'CAUSE'), ('Rotorua', 'GPE'), ('kills', 'EVENT')]


In [36]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /home/rangika/PythonCode/TrainedModel2.0


In [8]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
# Check the classes have loaded back consistently

test_text = 'Police rescue elderly person from Wellington Urban Motorway'

doc2 = nlp2(test_text)
for ent in doc2.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Loading from /home/rangika/PythonCode/TrainedModel2.0
Wellington 34 44 GPE


In [52]:
from tqdm import tqdm
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

    # get names of other pipes to disable them during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    # only train NER
with nlp.disable_pipes(*other_pipes) and warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

Losses {'ner': 50.021389760077}
Losses {'ner': 61.220083832740784}
Losses {'ner': 50.672827993286774}
Losses {'ner': 55.53044503927231}
Losses {'ner': 49.35659373732051}
Losses {'ner': 49.353583195068495}
Losses {'ner': 50.50948077440262}
Losses {'ner': 57.487945690751076}
Losses {'ner': 43.21051198942587}
Losses {'ner': 59.233455926470924}
Losses {'ner': 57.011119186878204}
Losses {'ner': 57.974149748682976}
Losses {'ner': 55.35906386375427}
Losses {'ner': 47.17279011383653}
Losses {'ner': 49.17373274639249}
Losses {'ner': 53.8401700258255}
Losses {'ner': 50.92825764417648}
Losses {'ner': 52.06319758296013}
Losses {'ner': 44.03205634839833}
Losses {'ner': 56.8142254948616}


In [7]:
# for text, _ in TRAIN_DATA:
#     doc = nlp(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [4]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

NameError: name 'nlp' is not defined

In [9]:
output_dir=Path("/home/rangika/PythonCode/TrainedModel1.0")

print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

# for text, _ in TRAIN_DATA:
#     doc = nlp2(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

test_text = 'Motorists heading home work Southern Motorway expect delays crash. The Transport Agency said crash blocking right southbound lane prior Southeastern Highway off-ramp. Pass scene care expect delays directions. SH1 2:45PM crash right southbound lane prior South Eastern Hwy off-ramp. Pass scene care expect delays directions. pic.twitter.comfEXiaHdLaa Akld Nthlnd November 14, 2019 statement police said attended multi-car nose tail crash, reported one seriously hurt. Advertisement couple people reported minor injuries, serious injuries involved, police spokesperson said. witness said counted five cars involved crash. Herald reporter Megan Harvey, driving south motorway crash happened, slam brakes avoid pile-up. went pretty much 100kmh stopped She said stopping safely, passed crash site saw five cars affected knock-on effect. The guy front stopped pretty fast, almost hit him, two missed it, Harvey said. The crash follows breakdown black Ferrari Southern Motorway 1pm. The Transport Agency said breakdown resulted two lanes blocked. People advised expect delays Wellington Symonds due breakdown, blocking lane three four adjacent Symonds off-ramp. Advertisement Tow services since removed Ferrari traffic still slow people advised consider using alternate routes delay journeys.'

doc = nlp2(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Loading from /home/rangika/PythonCode/TrainedModel1.0
Southern Motorway 28 45 LOC
The Transport Agency 67 87 ORG
Southeastern Highway 136 156 LOC
SH1 2:45PM 209 219 LOC
South Eastern Hwy 254 271 LOC
Akld Nthlnd 350 361 LOC
November 14, 2019 362 379 DATE
five 594 598 CARDINAL
Megan Harvey 636 648 PERSON
south motorway 658 672 LOC
slam 689 693 FAC
five 796 800 CARDINAL
two 883 886 CARDINAL
Harvey 898 904 ORG
Ferrari Southern Motorway 945 970 LOC
1pm 971 974 QUANTITY
The Transport Agency 976 996 ORG
two 1021 1024 CARDINAL
Wellington Symonds 1069 1087 LOC
three four 1117 1127 CARDINAL
Symonds off-ramp 1137 1153 LOC
Ferrari traffic 1196 1211 LOC


In [10]:
displacy.render(doc, style="ent")

In [31]:
import re
example = 'Tree fall in Spaghetti Junction woman in car.'
for match in re.finditer('Spaghetti Junction', example):
    print(match.start(), match.end())

13 31


In [12]:
import spacy 
  
nlp = spacy.load('en_core_web_sm') 
  
sentence = 'Seven people, including teenagers, have been taken to hospital after their car crashed in the mid-Canterbury town of Rakaia."'
doc = nlp(sentence) 
  
for ent in doc.ents: 
    print(ent.text, ent.start_char, ent.end_char, ent.label_) 

Seven 0 5 CARDINAL
mid-Canterbury 94 108 DATE
Rakaia 117 123 GPE


In [52]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Seven people, including teenagers, have been taken to hospital after their car crashed in the mid-Canterbury town of Rakaia.")
for token in doc:
    print(token,token.pos_)
    
# pos_tags = [(i, i.pos_) for i in doc]
# pos_tags

Seven NUM
people NOUN
, PUNCT
including VERB
teenagers NOUN
, PUNCT
have AUX
been AUX
taken VERB
to ADP
hospital NOUN
after ADP
their DET
car NOUN
crashed VERB
in ADP
the DET
mid ADJ
- ADJ
Canterbury ADJ
town NOUN
of ADP
Rakaia PROPN
. PUNCT


In [22]:
print("Verbs:", [token.text for token in doc if token.pos_ == "VERB"]) 

Verbs: ['backed', 'happened', 'westbound', 'asked', 'consider', 'using', 'following', 'remains', 'Consider', 'delaying', 'said', 'reopened', 'remained', 'link', 'Consider', 'delaying']


In [26]:
noun_adj_pairs = []
for i,token in enumerate(doc):
    if token.pos_ not in ('NOUN','VERB'):
        continue
    for j in range(i+1,len(doc)):
        if doc[j].pos_ == 'VERB':
            noun_adj_pairs.append((token,doc[j]))
            break
noun_adj_pairs

[(Traffic, backed),
 (backed, happened),
 (km, happened),
 (incident, happened),
 (incident, happened),
 (happened, westbound),
 (pm, westbound),
 (link, westbound),
 (westbound, asked),
 (Drivers, asked),
 (asked, consider),
 (avoid, consider),
 (area, consider),
 (consider, using),
 (using, following),
 (route, following),
 (link, following),
 (westbound, following),
 (following, remains),
 (incident, remains),
 (Traffic, remains),
 (remains, Consider),
 (northbound, Consider),
 (westbound, Consider),
 (Consider, delaying),
 (delaying, said),
 (delays, said),
 (said, reopened),
 (road, reopened),
 (reopened, remained),
 (traffic, remained),
 (remained, link),
 (link, Consider),
 (journey, Consider),
 (km, Consider),
 (Advertisement, Consider),
 (Consider, delaying)]

In [37]:
owners_possessions = [] 
for i in pos_tags:
    if i[1] == "VERB":
        owner = i[0].nbor(-1)
        possession = i[0].nbor(1)
        owners_possessions.append((owner, possession))
owners_possessions 

[(Traffic, 9),
 (incident, 12),
 (,, .),
 (Drivers, avoid),
 (area, using),
 (consider, alternative),
 (westbound, earlier),
 (Traffic, heavy),
 (., delaying),
 (Consider, journey),
 (Agency, 1.25pm),
 (road, traffic),
 (traffic, heavy),
 (Highway, journey),
 (Advertisement, delaying),
 (Consider, journey)]

In [46]:
#Dependancy Parcing
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")
doc = nlp("Seven people, including teenagers, have been taken to hospital after their car crashed in the mid-Canterbury town of Rakaia.")
a = []
b = []
c = []
d = []

for chunk in doc.noun_chunks:
    a.append(chunk.text)
    b.append(chunk.root.text)
    c.append(chunk.root.dep_)
    d.append(chunk.root.head.text)
    
df = pd.DataFrame({'a': a, 'b': b, 'c': c, 'd':d})
df

,a,b,c,d
0,Seven people,people,nsubjpass,taken
1,teenagers,teenagers,pobj,including
2,hospital,hospital,pobj,to
3,their car,car,nsubj,crashed
4,the mid-Canterbury town,town,pobj,in
5,Rakaia,Rakaia,pobj,of


In [48]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Seven people, including teenagers, have been taken to hospital after their car crashed in the mid-Canterbury town of Rakaia.")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Seven nummod people NOUN []
people nsubjpass taken VERB [Seven, ,, including, ,]
, punct people NOUN []
including prep people NOUN [teenagers]
teenagers pobj including VERB []
, punct people NOUN []
have aux taken VERB []
been auxpass taken VERB []
taken ROOT taken VERB [people, have, been, to, crashed, .]
to prep taken VERB [hospital]
hospital pobj to ADP []
after mark crashed VERB []
their poss car NOUN []
car nsubj crashed VERB [their]
crashed advcl taken VERB [after, car, in]
in prep crashed VERB [town]
the det town NOUN []
mid dep - ADJ []
- dep Canterbury ADJ [mid]
Canterbury amod town NOUN [-]
town pobj in ADP [the, Canterbury, of]
of prep town NOUN [Rakaia]
Rakaia pobj of ADP []
. punct taken VERB []


In [50]:
from spacy.symbols import nsubj, VERB

verbs = set()
for possible_subject in doc:
    if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)
print(verbs)

{crashed}
